In [62]:
import pandas as pd
import numpy as np

In [63]:
ptp = pd.read_csv('./PTP1B_Exp_Set/PTP1B_Ren_2011.csv')

In [64]:
ptp.head()

,SITE,FINAL TYR POS,Uniprot ID,Full_Seq
0,DSEGHLYTVPIR,14,Q03135,MSGGKYVDSEGHLYTVPIREQGNIYKPNNKAMADELSEKQVYDAHT...
1,TEPEPVYSMEAA,446,Q14247,MWKASAGHAVSIAQDDAGADDWETDPDFVNDVSEKEQRWGAKTVQG...
2,PPSSPIYEDAAP,421,Q14247,MWKASAGHAVSIAQDDAGADDWETDPDFVNDVSEKEQRWGAKTVQG...
3,ICTIDVYMIMVK,944,P07333,Not_Found
4,TAENAEYLRVAP,1197,P00533,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...


In [65]:
ptp['Dataset'] = 'PTP1B'

In [66]:
shp1 = pd.read_csv('./PTP1B_Exp_Set/SHP1_Ren_2011.csv')
shp2 = pd.read_csv('./PTP1B_Exp_Set/SHP2_Ren_2011.csv')

shp1['Dataset'] = 'SHP1'
shp2['Dataset'] = 'SHP2'

In [67]:
shp1.head()

,SITE,FINAL TYR POS,Uniprot ID,Full_Seq,Dataset
0,DDFDSDYENPDE,72,Q8WV28,MDKLNKITVPASQKLRQLQKMVHDIKNNEGGIMNKIKKLKVKAPPS...,SHP1
1,HSDSEMYVMPAE,84,Q8WV28,MDKLNKITVPASQKLRQLQKMVHDIKNNEGGIMNKIKKLKVKAPPS...,SHP1
2,ENADDSYEPPPV,96,Q8WV28,MDKLNKITVPASQKLRQLQKMVHDIKNNEGGIMNKIKKLKVKAPPS...,SHP1
3,lEDEADYVVPVE,178,Q8WV28,MDKLNKITVPASQKLRQLQKMVHDIKNNEGGIMNKIKKLKVKAPPS...,SHP1
4,EDNDENYIHPTE,189,Q8WV28,MDKLNKITVPASQKLRQLQKMVHDIKNNEGGIMNKIKKLKVKAPPS...,SHP1


In [68]:
shp2.head()

,SITE,FINAL TYR POS,Uniprot ID,Full_Seq,Dataset
0,VVDADEYLIPQQ,1016,P00533,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,SHP2
1,VSETDDYAEIID,397,Q05397,MAAAYLDPNLNHTPNSSTKTHLGTGMERSPGAMERVLKVFHYFESN...,SHP2
2,YMEDSTYYKASKG,576,Q05397,MAAAYLDPNLNHTPNSSTKTHLGTGMERSPGAMERVLKVFHYFESN...,SHP2
3,YMEDSTYYKASKG,577,Q05397,MAAAYLDPNLNHTPNSSTKTHLGTGMERSPGAMERVLKVFHYFESN...,SHP2
4,EADGELYVFNTP,285,Q13480,MSGGEVVCSGWLRKSPPEKKLKRYAWKRRWFVLRSGRLTGDPDVLE...,SHP2


In [69]:
allptp = pd.concat([ptp, shp1, shp2])

In [70]:
allptp['FINAL TYR POS'] = allptp['FINAL TYR POS'].astype(int)

In [71]:
fs_pep = []

for h,r in allptp.iterrows():
    fs = r['Full_Seq']
    pep = r['SITE']
    pos = r['FINAL TYR POS'] - 1
    
    if fs != 'Not_Found':
        fs_pep.append(fs[pos-6:pos+7])
    else:
        fs_pep.append('Not_Found')

In [72]:
allptp['Full_Seq_Peptide'] = fs_pep
allptp['FINAL TYR POS'] = allptp['FINAL TYR POS'] - 1

In [73]:
allptp[allptp['FINAL TYR POS'].isna()]

,SITE,FINAL TYR POS,Uniprot ID,Full_Seq,Dataset,Full_Seq_Peptide


In [74]:
# correct for 12AA long peptides that aren't present within the full sequence
allptp.loc[allptp['Full_Seq_Peptide'] == 'Not_Found', 'Full_Seq_Peptide'] = allptp.loc[allptp['Full_Seq_Peptide'] == 'Not_Found', 'SITE'] + 'A'

In [75]:
allptp = allptp.reset_index(drop=True)

In [76]:
allptp.loc[allptp['Full_Seq_Peptide'].str.len() != 13]

,SITE,FINAL TYR POS,Uniprot ID,Full_Seq,Dataset,Full_Seq_Peptide
35,AQENVDYVILKH,841,P20273,MHLLGPWLLLLVLEYLAFSDSSKWVFEHPETLYAWEGACVWIPCTY...,SHP1,AQENVDYVILKH
53,EDYSEYC,843,P15498,MELWRQCTHWLIQCRVLPPSHRVTWDGAQVCELAQALRDGVLLCQL...,SHP1,EEDYSEYC


In [77]:
allptp.loc[35,'Full_Seq_Peptide'] = 'AQENVDYVILKHA'
allptp.loc[53,'Full_Seq_Peptide'] = 'EEDYSEYCAAAAA'

In [78]:
allptp.loc[allptp['Full_Seq_Peptide'].str.len() != 13]

,SITE,FINAL TYR POS,Uniprot ID,Full_Seq,Dataset,Full_Seq_Peptide


In [79]:
# Read out full seqs to a fasta file, and peptides if not found
f = open('./PTP1B_Exp_Set/PTP1B_Musite_Deep_Run.fasta', 'w')

uids_written = []

for h,r in allptp.iterrows():
    fs = r['Full_Seq']
    pep = r['Full_Seq_Peptide']
    uid = r['Uniprot ID']
    
    if uid not in uids_written:
        f.write('>'+uid+'\n')
        if fs != 'Not_Found':
            f.write(fs+'\n')
        else:
            f.write(pep+'\n')
        uids_written.append(uid)
            
            
f.close()

In [80]:
# Read in MSD scores
msd = pd.read_csv('./PTP1B_Exp_Set/PTP1B_SHP1_SHP2_MSD.txt', sep='\t')

msd = msd[msd['Residue'].isna() == False]
msd['PTMscores'] = msd['PTMscores'].str[16:].astype(float)
msd['Position'] = msd['Position'].astype(int)
msd['Position'] = msd['Position'] - 1

msd['uid_pos'] = msd['ID'] + '_' + msd['Position'].astype(str)

In [81]:
allptp.loc[allptp['Full_Seq'] == 'Not_Found', 'FINAL TYR POS'] = 6

allptp['FINAL TYR POS'] = allptp['FINAL TYR POS'].astype(int)
allptp['uid_pos'] = allptp['Uniprot ID'] + '_' + allptp['FINAL TYR POS'].astype(str)

In [82]:
exp_with_msd = pd.merge(msd, allptp, on='uid_pos', indicator=True, how='right')

In [83]:
exp_with_msd[exp_with_msd['_merge'] != 'both']

,ID,Position,Residue,PTMscores,Cutoff=0.5,uid_pos,SITE,FINAL TYR POS,Uniprot ID,Full_Seq,Dataset,Full_Seq_Peptide,_merge
9,NaN,NaN,NaN,NaN,NaN,Q93052_6,SEGDTAYGQQVQ,6,Q93052,Not_Found,PTP1B,SEGDTAYGQQVQA,right_only
41,NaN,NaN,NaN,NaN,NaN,SHP-1_NoUID_6,KGQESEYGNITY,6,SHP-1_NoUID,Not_Found,SHP1,KGQESEYGNITYA,right_only


In [84]:
# Address three missing scores (ran manually)
exp_with_msd.loc[exp_with_msd['uid_pos'] == 'Q93052_6', 'PTMscores'] = 0.835
exp_with_msd.loc[exp_with_msd['uid_pos'] == 'SHP-1_NoUID_6', 'PTMscores'] = 0.921

In [95]:
final_exp = exp_with_msd[['uid_pos', 'FINAL TYR POS', 'Uniprot ID', 'Dataset', 'Full_Seq_Peptide', 'PTMscores']]

In [96]:
final_exp = final_exp.rename(columns={'FINAL TYR POS':'Position', 
                                      'Full_Seq_Peptide':'Peptide',
                                      'PTMscores':'SECONDARY_ML_SCORE'})

In [97]:
final_exp.to_csv('./PTP1B_Exp_Set/PTP1B_SHP1_SHP2_final_for_ML.csv')

In [98]:
final_exp

,uid_pos,Position,Uniprot ID,Dataset,Peptide,SECONDARY_ML_SCORE
0,Q03135_13,13,Q03135,PTP1B,DSEGHLYTVPIRE,0.967
1,Q14247_445,445,Q14247,PTP1B,TEPEPVYSMEAAD,0.876
2,Q14247_420,420,Q14247,PTP1B,LPSSPVYEDAASF,0.929
3,P07333_6,6,P07333,PTP1B,ICTIDVYMIMVKA,0.582
4,P00533_1196,1196,P00533,PTP1B,TAENAEYLRVAPQ,0.924
...,...,...,...,...,...,...
75,P78324_428,428,P78324,SHP2,DTNDITYADLNLP,0.944
76,P78324_452,452,P78324,SHP2,PNNHTEYASIQTS,0.919
77,P78324_469,469,P78324,SHP2,SEDTLTYADLDMV,0.932
78,O43609_52,52,O43609,SHP2,IRGSNEYTEGPSV,0.821


In [88]:
## GENERATE FEATURES!

In [89]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import rdmolfiles
import itertools
from collections import Counter

# Import file containing protdcal values (needed for feature generation later on)
protdcal = pd.read_csv('../protdcal_features.csv', index_col=0)

In [90]:
# METHOD FOR GENERATING FEATURES USING ONE SEQUENCE AT A TIME
def FeatureGen (sequence, protdcal):
    # FIRST: GENERATE PROTDCAL VALUES
    slist = list(sequence)   # first split sequence up into list
    # Go through sequence to get protdcal value
    pd = []
    for i in slist:
        pd.append(protdcal.loc[i].tolist())
    values = list(map(lambda *x: sum(x), *pd))   # add up values
    headers =  protdcal.columns.tolist()   # include headers


    # SECOND: GENERATE ONE-HOT ENCODING
    aa = ['K', 'R', 'H', 'A', 'I', 'L', 'M', 'V', 'F', 'W', 'Y', 'N', 'C', 'Q', 'S', 'T', 'D', 'E', 'G', 'P']   # possible amino acids
    # Make headers and one-hot encoding for each letter
    for i in aa:
        j = 0
        while j < len(sequence):
            headers.append('ONE-HOT_' + str(j) + '-' + i)  # make header
            if sequence[j] == i:
                values.append(1)
            else:
                values.append(0)
            j+=1


    # THIRD: GENERATE MACCS KEYS
    # Generate maccs keys
    mol = (rdmolfiles.MolFromFASTA(sequence))
    fp = (MACCSkeys.GenMACCSKeys(mol))
    maccs = fp.ToBitString()
    binary = list(maccs)   # split up into list
    values.extend(binary)   # add list onto resulting values
    # Generate headers for maccs keys
    mt = list(itertools.chain(range(len(binary))))
    mt = [str(s) + '_maccs' for s in mt]
    headers.extend(mt)   # append header values

    return values, headers

In [91]:
final_exp = final_exp.set_index('uid_pos', drop=True)

In [92]:
sequences = final_exp['Peptide']

In [93]:
## GENERATE FEATURES FOR OUR EXPERIMENTAL DATA
# Create df for results to go into
v, h = FeatureGen(sequences[0], protdcal)
features = pd.DataFrame(columns=h)
features.loc[len(features)] = v

i = 1

# Go through rest of sequences to generate feature set
while i < len(sequences):
    ts = sequences[i]
    value, header = FeatureGen(ts, protdcal)
    features.loc[len(features)] = value
    i+=1
    if i % 500 == 0:
        print(i, 'of', len(sequences), 'completed')

# Make the index the same as our initial dataframe
feat_x = features.set_index(final_exp.index)

# Isolate the methylated condition from the sequences as our y value
#feat_y = peptides['Experimental_Phosphorylation']

In [94]:
feat_x.to_csv('PTP1B_Exp_Set/PTP1B_SHP1_SHP2_final_for_ML_features.csv')